# Example of Weak Label Learning with warm up

In [4]:
from datasets import openml_datasets,torch_datasets

In [5]:
DS = openml_datasets.OpenML_Dataset('iris')

In [7]:
DS2 = torch_datasets.Torch_Dataset('mnist')

3.6%

100.0%


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



100.0%

Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw




33.8%

100.0%


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



100.0%

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw

